## Time Series Regression

Importing things we would need.

In [2]:
using FastAI, FastTimeSeries, Flux

### Loading The Data
We are going to select a dataset from the recently released Monash, UEA & UCR 
Time Series Extrinsic Regression Repository (2020) ([web](http://tseregression.org), [paper](https://arxiv.org/abs/2006.10996)). 

In [3]:
data, blocks = load(datarecipes()["appliances_energy"]);

`getobs` gets us a sample from the TimeSeriesDataset. It returns a tuple with the input time series and the correspodning label.

In [4]:
input, label = sample = getobs(data, 25)

(Float32[23.7 23.7 … 24.0 24.1; 30.856667 30.663334 … 31.066668 31.23; … ; 40.0 40.0 … 40.0 40.0; 0.6 0.55 … 1.3666667 1.3333334], 11.49f0)

Now we create a learning task for time-series regression. This means using the time-series to predict labels. We will use the TimeSeriesRow block as input and `Continuous` block as the target.

In [5]:
task = TSRegression(blocks, data);

The encodings passed in transform samples into formats suitable as inputs and outputs for a model

Let's check that samples from the created data container conform to the blocks of the learning task:

In [6]:
checkblock(task.blocks.sample, sample)

true

To get an overview of the learning task created, and as a sanity test, we can use describetask. This shows us what encodings will be applied to which blocks, and how the predicted ŷ values are decoded.

In [7]:
describetask(task)

**`SupervisedTask` summary**

Learning task for the supervised task with input `TimeSeriesRow` and target `Continuous`. Compatible with `model`s that take in `TimeSeriesRow` and output `Continuous`.

Encoding a sample (`encodesample(task, context, sample)`) is done through the following encodings:

|                  Encoding |              Name |      `blocks.input` | `blocks.target` |
| -------------------------:| -----------------:| -------------------:| ---------------:|
|                           | `(input, target)` |     `TimeSeriesRow` |    `Continuous` |
| `ContinuousPreprocessing` |                   |                     |                 |
|         `TSPreprocessing` |          `(x, y)` | **`TimeSeriesRow`** |                 |


In [8]:
encoded_sample = encodesample(task, Training(), sample)

(Float32[1.25567 1.25567 … 1.4426183 1.5049348; -2.3630838 -2.4116828 … -2.3102949 -2.269237; … ; 0.14111246 0.14111246 … 0.14111246 0.14111246; -0.7532581 -0.7651869 … -0.57035017 -0.5783027], Float32[11.49])

### Training

We will use an `InceptionTime` model for this task.

In [9]:
model = FastTimeSeries.Models.InceptionTime(24, 1);

We can `tasklossfn` to get a loss function suitable for our task.

In [10]:
lossfn = tasklossfn(task)

mse (generic function with 1 method)

Next we create a pair of training and validation data loaders. They take care of batching and loading the data in parallel in the background.

In [11]:
traindl, validdl = taskdataloaders(data, task, 8);

We will use an `ADAM` optimzer for this task.

In [12]:
optimizer = ADAM()

ADAM(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

With the addition of an optimizer and a loss function, we can now create a Learner and start training.

In [13]:
learner = Learner(model, lossfn; data=(traindl, validdl), optimizer=optimizer, callbacks = [ToGPU()]);

In [14]:
fitonecycle!(learner, 20, 0.03)

┌ Info: The GPU function is being called but the GPU is not accessible. 
│ Defaulting back to the CPU. (No action is required if you want to run on the CPU).
└ @ Flux /Users/saksham/.julia/packages/Flux/js6mP/src/functor.jl:192
Epoch 1 TrainingPhase(): 100%|██████████████████████████| Time: 0:01:40


┌───────────────┬───────┬─────────┐
│         Phase │ Epoch │    Loss │
├───────────────┼───────┼─────────┤
│ TrainingPhase │   1.0 │ 142.689 │
└───────────────┴───────┴─────────┘
┌─────────────────┬───────┬─────────┐
│           Phase │ Epoch │    Loss │
├─────────────────┼───────┼─────────┤
│ ValidationPhase │   1.0 │ 9526.84 │
└─────────────────┴───────┴─────────┘


Epoch 2 TrainingPhase(): 100%|██████████████████████████| Time: 0:00:02


┌───────────────┬───────┬─────────┐
│         Phase │ Epoch │    Loss │
├───────────────┼───────┼─────────┤
│ TrainingPhase │   2.0 │ 36.1776 │
└───────────────┴───────┴─────────┘
┌─────────────────┬───────┬─────────┐
│           Phase │ Epoch │    Loss │
├─────────────────┼───────┼─────────┤
│ ValidationPhase │   2.0 │ 2761.77 │
└─────────────────┴───────┴─────────┘


Epoch 3 TrainingPhase(): 100%|██████████████████████████| Time: 0:00:02


...

┌───────────────┬───────┬─────────┐
│         Phase │ Epoch │    Loss │
├───────────────┼───────┼─────────┤
│ TrainingPhase │  18.0 │ 10.1838 │
└───────────────┴───────┴─────────┘
┌─────────────────┬───────┬─────────┐
│           Phase │ Epoch │    Loss │
├─────────────────┼───────┼─────────┤
│ ValidationPhase │  18.0 │ 14.4063 │
└─────────────────┴───────┴─────────┘


Epoch 19 TrainingPhase(): 100%|█████████████████████████| Time: 0:00:03


┌───────────────┬───────┬─────────┐
│         Phase │ Epoch │    Loss │
├───────────────┼───────┼─────────┤
│ TrainingPhase │  19.0 │ 9.74364 │
└───────────────┴───────┴─────────┘
┌─────────────────┬───────┬────────┐
│           Phase │ Epoch │   Loss │
├─────────────────┼───────┼────────┤
│ ValidationPhase │  19.0 │ 14.059 │
└─────────────────┴───────┴────────┘


Epoch 20 TrainingPhase(): 100%|█████████████████████████| Time: 0:00:03


┌───────────────┬───────┬─────────┐
│         Phase │ Epoch │    Loss │
├───────────────┼───────┼─────────┤
│ TrainingPhase │  20.0 │ 8.15523 │
└───────────────┴───────┴─────────┘
┌─────────────────┬───────┬─────────┐
│           Phase │ Epoch │    Loss │
├─────────────────┼───────┼─────────┤
│ ValidationPhase │  20.0 │ 15.6445 │
└─────────────────┴───────┴─────────┘


We can save the model for later inference using `savetaskmodel`

In [15]:
savetaskmodel("tsregression.jld2", task, learner.model; force = true)